loading required libraries

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 5000)
from dotenv import load_dotenv
import json
from src.pdf import create_pdf
from src.gpt import evaluate_gpt
from src.utils import (
    load_preprocess_data,
    extract_all_diseases_per_department,
    filterDepartment,
    getDepartmentStatistics,
    run_prediction)
load_dotenv()
from prompts import single_shot_disease_only_prompt,open_ended__top4_prompt,prompt_template
from src.gpt import doctor_prompt_gpt
from src.utils import select_case_components

importing the dataset

In [2]:
filePath="dataset/clinicallab/data_en.json"
df=load_preprocess_data(filePath)


number of total cases are 1500

each case have the following fields ['id', 'clinical_case_uid', 'language', 'clinical_department', 'principal_diagnosis', 'preliminary_diagnosis', 'diagnostic_basis', 'differential_diagnosis', 'treatment_plan', 'clinical_case_summary', 'imageological_examination', 'laboratory_examination', 'pathological_examination', 'therapeutic_principle']

number of departments available are 24

 all the departments available are
clinical_department
orthopedics department                              100
anus and intestine surgical department              100
hepatobiliary and pancreas surgical department       99
urinary surgical department                          90
endocrinology department                             80
gynecology department                                80
otolaryngology head and neck surgical department     80
neurology department                                 80
thoracic surgical department                         70
respiratory medicine de

filtering the dataset

In [3]:
department="neurology department"
departmentdf=filterDepartment(df,department)
# departmentdf.to_json(f"dataset/clinicallab/department/{department}.json", orient='records', lines=False, indent=4)
getDepartmentStatistics(departmentdf)

number of principal diagnosis are 8
principal_diagnosis
parkinson disease            10
epilepsy                     10
transient ischemic attack    10
myelitis                     10
cerebral infarction          10
subarachnoid hemorrhage      10
alzheimer disease            10
facial neuritis              10
Name: count, dtype: int64
number of preliminary_diagnosis are 8


In [ ]:
# Function to extract disease names from a single row


In [4]:
uniqueDiseases=extract_all_diseases_per_department(departmentdf)
print(len(uniqueDiseases))

number of unique diseases are 58
58
['acute myelitis', 'acute spinal cord compression', 'acute spinal cord compression syndrome', 'alzheimer disease', 'benign positional vertigo', 'brain hemorrhage', 'brainstem infarction', 'cerebral hemorrhage', 'cerebral infarction', 'cervical vertigo', 'cranial tumor', 'dementia with lewy bodies (dlb)', 'diabetic neuropathy', 'electrolyte disorder', 'epilepsy', 'facial neuritis', 'frontotemporal dementia', 'frontotemporal dementia (ftd)', 'guillain-barre syndrome', 'guillain-barré syndrome', 'hereditary spastic paraplegia', 'intracranial infection', 'intracranial space-occupying lesions', 'lewy body dementia', 'lyme disease', 'multiple sclerosis', 'multiple system atrophy', 'myelitis', 'neurological lyme disease', 'neuromyelitis optica', 'optic neuritis', 'otic origin facial nerve paralysis', 'parkinson disease', "parkinson's disease dementia (pdd)", 'periodic paralysis', 'polymyositis', 'posterior fossa lesions', 'posterior fossa tumor or meningiti

In [5]:
uniqueDiseases1=['Lumbar Tuberculosis', 'systemic lupus erythematosus', 'Low T3 Syndrome', 'ankylosing spondylitis', "Graves' disease", 'Diabetes insipidus', 'Psoriatic Arthritis', 'Type 1 Diabetes', 'Type 1 diabetes', 'Hyperthyroidism', 'Scleredema', 'hypothyroidism', 'Hypoglycemia', 'Hypokalemia', 'Myositis', 'subacute thyroiditis', 'Simple Skin Allergy', 'Diabetes Insipidus', 'Simple Goiter', 'Psoriatic arthritis', 'rheumatoid arthritis', 'Rheumatic Fever', 'diabetes mellitus', 'Simple goiter', 'Colorectal cancer', 'gouty arthritis', 'Rheumatic fever', 'Osteoarthritis']
uniqueDiseases1=['acute suppurative thyroiditis', 'ankylosing spondylitis', 'cervical and lumbar disc herniation', 'chronic lymphocytic thyroiditis', 'colorectal cancer', 'diabetes insipidus', 'diabetes mellitus', 'gouty arthritis', "graves' disease", "hashimoto's thyroiditis",'hyperthyroidism', 'hypoglycemia', 'hypokalemia', 'hypothyroidism', 'inflammatory bowel disease', 'lada', 'localized scleroderma', 'low t3 syndrome', 'lumbar tuberculosis', 'mixed connective tissue disease', 'myositis', 'osteoarthritis', 'psoriatic arthritis', 'rheumatic fever', 'rheumatoid arthritis', 'scleredema', 'simple goiter', 'simple skin allergy', 'subacute thyroiditis', 'systemic lupus erythematosus', 'type 1 diabetes']
uniqueDiseases2=['Brain Hemorrhage', 'parkinson disease', 'Epilepsy', 'Cerebral Infarction', 'Diabetic Neuropathy', 'Cerebral hemorrhage', 'Lewy Body Dementia', 'Cervical vertigo', 'Tumorous Stroke', 'alzheimer disease', 'myelitis', 'Lyme Disease', 'Multiple sclerosis', 'Acute Myelitis', 'Tumoral Stroke', 'Optic neuritis', 'subarachnoid hemorrhage', 'Tumor Stroke', 'Cranial tumor', 'Vascular Dementia', 'Lyme disease', 'Syncope', 'Pseudoseizures', 'Periodic Paralysis', 'Primary Tremor', 'Diabetic neuropathy', 'facial neuritis', 'Polymyositis', 'transient ischemic attack']
uniqueDiseasesall=[uniqueDiseases1,uniqueDiseases2]
print(len(uniqueDiseases))

58


In [6]:
departments=["nephrology department",
             "gynecology department",
             "endocrinology department", 
             "neurology department",
             "pediatrics department",
             "cardiac surgical department",                          
             "gastrointestinal surgical department",
             "respiratory medicine department",]

models = ["llama3.1", "gemma2", "phi3:14b", "mistral-nemo"]



In [7]:
def run_prediction(df,prompt,departments,models=[],type="semi_ended",laboratory_examination="result",image_examination="findings",skip=6):
    required_fields=[ "Patient basic information",
                 "Chief complaint",
                 "Medical history",
                 "Physical examination",
                 "Laboratory examination",
                 "Imageological examination",
                 "Auxillary examination",
                 "Pathological examination"
    
]
    report_type=f"{laboratory_examination}_{image_examination}"
    for i  in range(len(departments)):
        department=departments[i]
        results={}
        print("department is",department)
        departmentdf=filterDepartment(df,department)
        caseNumbers = [i for i in range(1, len(departmentdf), skip)]
        print(caseNumbers)
        getDepartmentStatistics(departmentdf)
        for caseNumber in caseNumbers:
            case_details={}        
            case_id,principal_diagnosis,differential_diagnosis,clinical_case_dict,filtered_clinical_case_dict=select_case_components(departmentdf,caseNumber,required_fields,laboratory_examination,image_examination)
            case_details["original"]={"main-diagnosis":principal_diagnosis,"differential_diagnosis":differential_diagnosis}
            print("case_id",case_id)
            print("principal diagnosis",principal_diagnosis)
            output0=doctor_prompt_gpt(prompt,filtered_clinical_case_dict,"gpt-4",differential_diagnosis,department)
            
            case_details["gpt-4"]=output0
            # case_details["llama3.1"]=output1
            # case_details["gemma2"]=output2
            results[str(case_id)]=case_details
        with open(f"{department}_{type}_{report_type}.json", "w") as outfile: 
            json.dump(results, outfile)

In [11]:
departments=[#"nephrology department",
            #  "gynecology department",
            #  "endocrinology department", 
            "neurology department",
            #  "pediatrics department",
            #  "cardiac surgical department",                          
            #  "gastrointestinal surgical department",
            #  "respiratory medicine department",
            ]
type="prompt_template"
prompt=prompt_template
type="single_shot"
prompt=single_shot_disease_only_prompt
type="open_ended"
prompt=open_ended__top4_prompt# 
run_prediction(df,prompt,departments,type=type,skip=8)

department is neurology department
[1, 9, 17, 25, 33, 41, 49, 57, 65, 73]
number of principal diagnosis are 8
principal_diagnosis
parkinson disease            10
epilepsy                     10
transient ischemic attack    10
myelitis                     10
cerebral infarction          10
subarachnoid hemorrhage      10
alzheimer disease            10
facial neuritis              10
Name: count, dtype: int64
number of preliminary_diagnosis are 8
case_id 712
principal diagnosis parkinson disease


case_id 720
principal diagnosis parkinson disease
case_id 728
principal diagnosis epilepsy
case_id 736
principal diagnosis transient ischemic attack
case_id 744
principal diagnosis myelitis
case_id 752
principal diagnosis cerebral infarction
case_id 760
principal diagnosis cerebral infarction
case_id 768
principal diagnosis subarachnoid hemorrhage
case_id 776
principal diagnosis alzheimer disease
case_id 784
principal diagnosis facial neuritis


In [ ]:
create_pdf(departments,type)

In [ ]:
file_path="neurology department_semi_ended_result_findings.json"
with open(file_path, "r") as file:
    data = json.load(file)

In [ ]:
# Initialize results dictionary
results = {model: {"true": [], "false": [], "count_true": 0, "count_false": 0} for model in ["gpt-4", "llama3.1", "gemma2"]}

# Iterate through each case in the data
for caseNumber, case in data.items():
    ground_truth_disease = case["original"]["main-diagnosis"]
    
    # Evaluate predictions for each model
    for model in results.keys():
        predicted = case[model]
        output = list(evaluate_gpt(ground_truth_disease, predicted))
        result=str(output[1])
        # Append case number to correct or false lists and increment counts
        results[model][result.lower()].append(caseNumber)
        results[model][f'count_{result.lower()}'] += 1

# Output results
print(results)
